In [4]:
class Matrix:
    def __init__(self, matrix):
        self.matrix = matrix

    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.matrix])

    def _elementwise_operation(self, other, operator):# Вспомогательная функция для выполняемых операций, принимает другую матрицу или число и оператор
        if isinstance(other, (int, float)):
            result = [[operator(x, other) for x in row] for row in self.matrix]# Если other - число, выполняем операцию поэлементно для каждого элемента матрицы
        elif isinstance(other, Matrix):
            if self.shape() != other.shape():# Если other - объект класса Matrix, проверяем, что размерности матриц совпадают
                raise ValueError("Размеры матрицы должны быть одинаковыми")
            result = [[operator(x, y) for x, y in zip(row1, row2)] for row1, row2 in zip(self.matrix, other.matrix)]# Выполняем операцию поэлементно для каждой пары элементов двух матриц
        else:
            raise TypeError("Неподдерживаемый тип")# Выбрасываем исключения если other не число и не матрица
        return Matrix(result)

    def __add__(self, other):
        return self._elementwise_operation(other, lambda x, y: x + y)

    def __sub__(self, other):
        return self._elementwise_operation(other, lambda x, y: x - y)

    def __truediv__(self, other):
        return self._elementwise_operation(other, lambda x, y: x / y if y != 0 else 0)

    def __mul__(self, other):
        return self._elementwise_operation(other, lambda x, y: x * y)

    def __matmul__(self, other):
        if self.shape()[1] != other.shape()[0]:
            raise ValueError("Количество столбцов в первой матрице должно быть равно количеству строк во второй матрице")
        result = [[sum(a * b for a, b in zip(row1, col2)) for col2 in zip(*other.matrix)] for row1 in self.matrix]
        return Matrix(result)

    def shape(self): 
        return (len(self.matrix), len(self.matrix[0]))# Возвращает размерность матрицы в виде кортежа (количество строк, количество столбцов)

    def T(self):
        result = [[x for x in row] for row in zip(*self.matrix)]
        return Matrix(result)


In [5]:
%pip install -q ipytest pytest

import numpy as np
import ipytest
import pytest
ipytest.autoconfig()

Note: you may need to restart the kernel to use updated packages.


In [6]:
%%ipytest

matrix1 = Matrix([[1,2,3],[4,5,6],[7,8,9]])
matrix2 = Matrix([[10,11,12],[13,14,15],[16,17,18]])

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])
def test_matrix_add_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (np.array((matrix1 + matrix2).matrix) == (np.array(matrix1.matrix) + np.array(matrix2.matrix))).all()
    ), 'add section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 13.2]])
def test_matrix_add_number(matrix1: Matrix, number: int | float):
    assert (
        (np.array((matrix1 + number).matrix) == np.array(matrix1.matrix) + number).all()
    ), 'add section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])
def test_matrix_sub_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (np.array((matrix1 - matrix2).matrix) == np.array(matrix1.matrix) - np.array(matrix2.matrix)).all()
    ), 'sub section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 12.2]])    
def test_matrix_sub_number(matrix1: Matrix, number: int | float):
    assert (
        (np.array((matrix1 - number).matrix) == np.array(matrix1.matrix) - number).all()
    ), 'sub section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1", [matrix1])    
def test_matrix_transpose(matrix1: Matrix):
    assert (
        (np.array(matrix1.T().matrix) == np.array(matrix1.matrix).T).all()
    ), 'transpose section is not working right'
    
# -----------------------------------------

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])
def test_matrix_mul_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (np.array((matrix1 * matrix2).matrix) == np.array(matrix1.matrix) * np.array(matrix2.matrix)).all()
    ), 'mul section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 13]])
def test_matrix_mul_number(matrix1: Matrix, number: int | float):
    assert (
        (np.array((matrix1 * number).matrix) == np.array(matrix1.matrix) * number).all()
    ), 'mul section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2]])    
def test_matrix_div_matrix(matrix1: Matrix, matrix2: Matrix):
    assert (
        (np.array((matrix1 / matrix2).matrix) == np.array(matrix1.matrix) / np.array(matrix2.matrix)).all()
    ), 'div section with matrices is not working right'

@pytest.mark.parametrize("matrix1,number", [[matrix1, 2]])    
def test_matrix_div_number(matrix1: Matrix, number: int | float):
    assert (
        (np.array((matrix1 / number).matrix) == np.array(matrix1.matrix) / number).all()
    ), 'div section with matrix and number is not working right'

@pytest.mark.parametrize("matrix1,matrix2", [[matrix1, matrix2.T()]])    
def test_matrix_matmul_matrix(matrix1: Matrix, matrix2: Matrix):
    assert(
        (np.array((matrix1 @ matrix2).matrix) == np.array(matrix1.matrix) @ np.array(matrix2.matrix)).all()
    ), 'matmul section is not working right, and don\'t forget about T() if needed'

..........                                                                                   [100%]
10 passed in 0.09s
